In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# import matplotlib to show plots inline.
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import os

import astropy.units as u
import matplotlib as mpl
import sys 
sys.path.append("../../")
from plot_OpSims import plot_OpSims_Nqso_hist

In [ ]:
# import maf python modules
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles

In [ ]:
# import convenience functions
import sys 
sys.path.append("../../../LSST_OpSim")
from Scripts_NBs.opsimUtils import *

In [ ]:
NSIDE=64
your_username = "rjassef"
folder_mafoutput = "EM5_{0:d}".format(NSIDE)
resultDbPath = '/home/idies/workspace/Temporary/{0}/scratch/MAFOutput/{1}'.format(
    your_username, folder_mafoutput)
metricDataPath = '/home/idies/workspace/Temporary/{0}/scratch/MAFOutput/{1}/MetricData/'.format(
    your_username, folder_mafoutput)

In [ ]:
# get a dictionary of resultDb from given directory
resultDbs = getResultsDbs(resultDbPath)

In [ ]:
# retrieve metricBundles for each opsim run and store them in a dictionary
bundleDicts_raw = dict()
for runName in resultDbs:
    bundleDicts_raw[runName] = bundleDictFromDisk(resultDbs[runName], runName, metricDataPath)

In [ ]:
#Rearrange the bundleDicts_raw dictionary so that the keys are always the same as for the first run.
dbRuns = list(bundleDicts_raw.keys())
Keys = list(bundleDicts_raw[dbRuns[0]].keys())
bundleDicts = dict()
for runName in dbRuns:
    bundleDicts[runName] = dict()
    Keys_raw = bundleDicts_raw[runName]
    for Key in Keys:
        for Key_raw in Keys_raw:
            if Key[1]==Key_raw[1]:
                bundleDicts[runName][Key] = bundleDicts_raw[runName][Key_raw]

In [ ]:
# check keys
dbRuns = list(resultDbs.keys())
bd_keys = list(bundleDicts[dbRuns[0]].keys())
print(bd_keys)

In [ ]:
sys.path.append("../")
from Fast_Nqso import Fast_Nqso

In [ ]:
zmin = 0.3
zmax = 6.7
mbright = 15.7
for run in dbRuns:
    print(run)
    for key in bundleDicts[run].keys():
        mask  = bundleDicts[run][key].metricValues.mask
        mlim5 = bundleDicts[run][key].metricValues[~mask]
        qso_counter = Fast_Nqso("LSSTi","Shen20","A",area=bundleDicts[run][key].slicer.pixArea*u.sr)
        print(key, np.sum(qso_counter.Nqso(zmin, zmax, mbright, mlim5)))